# Importations and Required Libraries

In [1]:
pip install ipython-sql sqlalchemy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


**load SQL magic and set up an SQLite database**

In [5]:
%load_ext sql
import sqlite3
import pandas as pd
from sqlalchemy import create_engine


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


# 1: EXTRACTION
**a: Load Data into Pandas**

In [6]:
# Load dataset files into DataFrames
df_active = pd.read_csv("Flipside Dataset/flipside_days_active.csv")
df_labels = pd.read_csv("Flipside Dataset/flipside_dim_labels.csv", delimiter=",", on_bad_lines="skip")
df_new_programs = pd.read_csv("Flipside_Dataset_2/Solana_Weekly_New_Programs.csv")
df_new_users = pd.read_csv("Flipside_Dataset_2/Solana Weekly New Users.csv")
df_unique_programs = pd.read_csv("Flipside_Dataset_2/Solana_Weekly_Unique_Programs.csv")
df_unique_users = pd.read_csv("Flipside_Dataset_2/Solana_Weekly_Unique_Users.csv")
df_days_since_last_use = pd.read_csv("Flipside Dataset/weekly_days_since_last_use.csv")
df_users_last_use = pd.read_csv("Flipside_Dataset_2/Solana_last_use_data.csv")

**b: create a database connection**

In [7]:
engine = create_engine('sqlite:///flipside_data.db')  
conn = engine.connect()

**c: Store DataFrames as SQL Tables**

In [8]:
df_active.to_sql('flipside_days_active', conn, if_exists='replace', index=False)
df_labels.to_sql('flipside_dim_labels', conn, if_exists='replace', index=False)
df_new_programs.to_sql('solana_weekly_new_programs', conn, if_exists='replace', index=False)
df_new_users.to_sql('solana_weekly_new_users', conn, if_exists='replace', index=False)
df_unique_programs.to_sql('solana_weekly_unique_programs', conn, if_exists='replace', index=False)
df_unique_users.to_sql('solana_weekly_unique_users', conn, if_exists='replace', index=False)
df_days_since_last_use.to_sql('weekly_days_since_last_use', conn, if_exists='replace', index=False)
df_users_last_use.to_sql('solana_last_use_data', conn, if_exists='replace', index=False)


100001

**d: Query using %%sql to inspect the tables**

In [9]:
%%sql 
sqlite:///flipside_data.db
SELECT * 
FROM solana_weekly_new_users 
LIMIT 5;

Done.


WEEK,NEW_USERS
2025-03-03 00:00:00.000,3.0
2025-02-03 00:00:00.000,1.0
2025-01-20 00:00:00.000,1.0
2025-01-13 00:00:00.000,2.0
2025-01-06 00:00:00.000,2.0


In [10]:
%%sql 
sqlite:///flipside_data.db
SELECT * 
FROM weekly_days_since_last_use 
LIMIT 5;

Done.


CREATION_DATE,Days since last use,Days since creation
2020-10-07,633.6165674428066,750.5566283125927
2020-10-14,611.4229036788537,743.5566283125927
2020-10-21,528.2639841669929,736.5566283125927
2020-10-28,492.9001518006344,729.5566283125927
2020-11-04,696.7685171327714,722.5566283125927


# Merge Tables 


**I. Merge flipside_days_active, weekly_days_since_last_use, and solana_last_use_data on the ADDRESS column.**

In [13]:
%%sql
ALTER TABLE flipside_days_active RENAME COLUMN SIGNER TO ADDRESS;

SELECT 
    fda.ADDRESS, 
    fda.FIRST_TX_DATE, 
    fda.FIRST_PROGRAM_ID, 
    fda.LAST_TX_DATE, 
    fda.LAST_PROGRAM_ID, 
    fda.NUM_DAYS_ACTIVE, 
    fda.NUM_TXS, 
    fda.TOTAL_FEES, 
    fda.PROGRAMS_USED,
    wds.CREATION_DATE, 
    wds."Days since last use", 
    wds."Days since creation", 
    sld.LAST_USE
FROM flipside_days_active fda
LEFT JOIN weekly_days_since_last_use wds ON fda.ADDRESS = wds.CREATION_DATE
LEFT JOIN solana_last_use_data sld ON fda.ADDRESS = sld.ADDRESS
LIMIT 5; 


 * sqlite:///flipside_data.db
Done.
Done.


ADDRESS,FIRST_TX_DATE,FIRST_PROGRAM_ID,LAST_TX_DATE,LAST_PROGRAM_ID,NUM_DAYS_ACTIVE,NUM_TXS,TOTAL_FEES,PROGRAMS_USED,CREATION_DATE,Days since last use,Days since creation,LAST_USE
9P65j1MLdkve3iShPXF3H6F32qGhKYmVLyza5J4ZvkyU,2025-03-11 00:00:00.000,11111111111111111111111111111111,2025-03-11 00:00:00.000,HDwcJBJXjL9FpJ7UBsYBtaDjsBUhuLCUYoz3zr8SWWaQ,1.0,1.0,None,"[""11111111111111111111111111111111"",""11111111111111111111111111111111"",""HDwcJBJXjL9FpJ7UBsYBtaDjsBUhuLCUYoz3zr8SWWaQ"",""HDwcJBJXjL9FpJ7UBsYBtaDjsBUhuLCUYoz3zr8SWWaQ""]",None,None,None,None
6VvDGnnAeEeK3yd4jAdH5X7ZdRVTsrbrg4beDZvQemmG,2025-03-11 00:00:00.000,11111111111111111111111111111111,2025-03-11 00:00:00.000,675kPX9MHTjS2zt1qfr1NYHuzeLXfQM9H24wFSUt1Mp8,1.0,1.0,None,"[""11111111111111111111111111111111"",""11111111111111111111111111111111"",""675kPX9MHTjS2zt1qfr1NYHuzeLXfQM9H24wFSUt1Mp8""]",None,None,None,None
gGf97pe7apxKNmQyiBctckiLeuPTBrbh4HridK6bNBz,2025-03-11 00:00:00.000,11111111111111111111111111111111,2025-03-11 00:00:00.000,675kPX9MHTjS2zt1qfr1NYHuzeLXfQM9H24wFSUt1Mp8,1.0,1.0,None,"[""11111111111111111111111111111111"",""11111111111111111111111111111111"",""675kPX9MHTjS2zt1qfr1NYHuzeLXfQM9H24wFSUt1Mp8""]",None,None,None,None
7VEETqsSS9QdhZZg2fZcpivzLYg8zbQrKEzpFoN9zpav,2025-03-11 00:00:00.000,11111111111111111111111111111111,2025-03-11 00:00:00.000,675kPX9MHTjS2zt1qfr1NYHuzeLXfQM9H24wFSUt1Mp8,1.0,1.0,None,"[""11111111111111111111111111111111"",""11111111111111111111111111111111"",""675kPX9MHTjS2zt1qfr1NYHuzeLXfQM9H24wFSUt1Mp8""]",None,None,None,None
ESJkFSHeeHE4rBHKxVAKA4QxjL93KPH7MbWrwkyDQgQx,2025-03-11 00:00:00.000,11111111111111111111111111111111,2025-03-11 00:00:00.000,675kPX9MHTjS2zt1qfr1NYHuzeLXfQM9H24wFSUt1Mp8,1.0,2.0,10000.0,"[""11111111111111111111111111111111"",""675kPX9MHTjS2zt1qfr1NYHuzeLXfQM9H24wFSUt1Mp8""]",None,None,None,None


In [22]:
%%sql
SELECT * FROM weekly_days_since_last_use LIMIT 5;
SELECT * FROM solana_last_use_data LIMIT 5;


 * sqlite:///flipside_data.db
Done.
Done.


ADDRESS,LAST_USE
GtYVsyLCGKx29cZtTfvtdcZ3rwamYKan4FpZ286Yg5A,2025-03-17 00:00:00.000
HPPWqvpsU7PAzunxQW4gmNiPJDiuzdLodogKAz58sUmd,2025-03-17 00:00:00.000
BcqQYWEhqE3Qo3RRMF72Dsc7t65Tm7zSEd98EgHXb4oz,2025-03-17 00:00:00.000
DJo8CG7sJd7dW23Dk6NJvRFZNfMUoioaLxqQ9V554jSD,2025-03-17 00:00:00.000
AGkfxH1QfxxbwZSNuWe1pwCp6yBkRz8TNbKmSUZrQvWz,2025-03-17 00:00:00.000


**Merge solana_weekly_new_users and solana_weekly_unique_users since they track newly created users and unique occurrences.**

In [14]:
%%sql
SELECT 
    COALESCE(nu.WEEK, uu.WEEK) AS WEEK, 
    nu.NEW_USERS, 
    uu.UNIQUE_USERS
FROM solana_weekly_new_users nu
LEFT JOIN solana_weekly_unique_users uu 
ON nu.WEEK = uu.WEEK

UNION

SELECT 
    COALESCE(nu.WEEK, uu.WEEK) AS WEEK, 
    nu.NEW_USERS, 
    uu.UNIQUE_USERS
FROM solana_weekly_unique_users uu
LEFT JOIN solana_weekly_new_users nu 
ON uu.WEEK = nu.WEEK
WHERE nu.WEEK IS NULL
LIMIT 10;


 * sqlite:///flipside_data.db
Done.


WEEK,NEW_USERS,UNIQUE_USERS
2023-08-28 00:00:00.000,1.0,None
2023-09-04 00:00:00.000,1.0,None
2023-09-11 00:00:00.000,1.0,None
2023-09-18 00:00:00.000,1.0,None
2023-10-30 00:00:00.000,1.0,None
2023-11-13 00:00:00.000,4.0,None
2023-12-11 00:00:00.000,1.0,None
2024-01-22 00:00:00.000,3.0,None
2024-01-29 00:00:00.000,2.0,33832569.0
2024-02-05 00:00:00.000,3.0,340112.0


In [19]:
# Connect to database
DATABASE_URL = "sqlite:///flipside_data.db"
engine = create_engine(DATABASE_URL)

# Run SQL query and load into Pandas DataFrame
query = """
    

    SELECT 
        fda.ADDRESS, 
        fda.FIRST_TX_DATE, 
        fda.FIRST_PROGRAM_ID, 
        fda.LAST_TX_DATE, 
        fda.LAST_PROGRAM_ID, 
        fda.NUM_DAYS_ACTIVE, 
        fda.NUM_TXS, 
        fda.TOTAL_FEES, 
        fda.PROGRAMS_USED,
        wds.CREATION_DATE, 
        wds."Days since last use", 
        wds."Days since creation", 
        sld.LAST_USE
    FROM flipside_days_active fda
    LEFT JOIN weekly_days_since_last_use wds ON fda.ADDRESS = wds.CREATION_DATE
    LEFT JOIN solana_last_use_data sld ON fda.ADDRESS = sld.ADDRESS
    LIMIT 5;
"""

# Execute SQL query and convert to DataFrame
df = pd.read_sql(query, engine)

# Display the first few rows
print(df.head())
print(type(df))

                                        ADDRESS            FIRST_TX_DATE  \
0  9P65j1MLdkve3iShPXF3H6F32qGhKYmVLyza5J4ZvkyU  2025-03-11 00:00:00.000   
1  6VvDGnnAeEeK3yd4jAdH5X7ZdRVTsrbrg4beDZvQemmG  2025-03-11 00:00:00.000   
2   gGf97pe7apxKNmQyiBctckiLeuPTBrbh4HridK6bNBz  2025-03-11 00:00:00.000   
3  7VEETqsSS9QdhZZg2fZcpivzLYg8zbQrKEzpFoN9zpav  2025-03-11 00:00:00.000   
4  ESJkFSHeeHE4rBHKxVAKA4QxjL93KPH7MbWrwkyDQgQx  2025-03-11 00:00:00.000   

                   FIRST_PROGRAM_ID             LAST_TX_DATE  \
0  11111111111111111111111111111111  2025-03-11 00:00:00.000   
1  11111111111111111111111111111111  2025-03-11 00:00:00.000   
2  11111111111111111111111111111111  2025-03-11 00:00:00.000   
3  11111111111111111111111111111111  2025-03-11 00:00:00.000   
4  11111111111111111111111111111111  2025-03-11 00:00:00.000   

                                LAST_PROGRAM_ID  NUM_DAYS_ACTIVE  NUM_TXS  \
0  HDwcJBJXjL9FpJ7UBsYBtaDjsBUhuLCUYoz3zr8SWWaQ              1.0      1.0   
1  6

In [20]:
print(df.head())  # View first few rows
print(df.dtypes)  # Check column types


                                        ADDRESS            FIRST_TX_DATE  \
0  9P65j1MLdkve3iShPXF3H6F32qGhKYmVLyza5J4ZvkyU  2025-03-11 00:00:00.000   
1  6VvDGnnAeEeK3yd4jAdH5X7ZdRVTsrbrg4beDZvQemmG  2025-03-11 00:00:00.000   
2   gGf97pe7apxKNmQyiBctckiLeuPTBrbh4HridK6bNBz  2025-03-11 00:00:00.000   
3  7VEETqsSS9QdhZZg2fZcpivzLYg8zbQrKEzpFoN9zpav  2025-03-11 00:00:00.000   
4  ESJkFSHeeHE4rBHKxVAKA4QxjL93KPH7MbWrwkyDQgQx  2025-03-11 00:00:00.000   

                   FIRST_PROGRAM_ID             LAST_TX_DATE  \
0  11111111111111111111111111111111  2025-03-11 00:00:00.000   
1  11111111111111111111111111111111  2025-03-11 00:00:00.000   
2  11111111111111111111111111111111  2025-03-11 00:00:00.000   
3  11111111111111111111111111111111  2025-03-11 00:00:00.000   
4  11111111111111111111111111111111  2025-03-11 00:00:00.000   

                                LAST_PROGRAM_ID  NUM_DAYS_ACTIVE  NUM_TXS  \
0  HDwcJBJXjL9FpJ7UBsYBtaDjsBUhuLCUYoz3zr8SWWaQ              1.0      1.0   
1  6

In [12]:
%%sql
WITH first_week AS (
    SELECT 
        ADDRESS, 
        MIN(WEEK) AS signup_week
    FROM solana_weekly_new_users
    GROUP BY ADDRESS
),
weekly_retention AS (
    SELECT 
        f.signup_week, 
        w.WEEK, 
        COUNT(DISTINCT w.ADDRESS) AS returning_users
    FROM first_week f
    JOIN solana_weekly_unique_users w 
    ON f.ADDRESS = w.ADDRESS AND w.WEEK >= f.signup_week
    GROUP BY f.signup_week, w.WEEK
)
SELECT 
    signup_week, 
    WEEK, 
    returning_users 
FROM weekly_retention
ORDER BY signup_week, WEEK;


 * sqlite:///flipside_data.db
(sqlite3.OperationalError) no such column: ADDRESS
[SQL: WITH first_week AS (
    SELECT 
        ADDRESS, 
        MIN(WEEK) AS signup_week
    FROM solana_weekly_new_users
    GROUP BY ADDRESS
),
weekly_retention AS (
    SELECT 
        f.signup_week, 
        w.WEEK, 
        COUNT(DISTINCT w.ADDRESS) AS returning_users
    FROM first_week f
    JOIN solana_weekly_unique_users w 
    ON f.ADDRESS = w.ADDRESS AND w.WEEK >= f.signup_week
    GROUP BY f.signup_week, w.WEEK
)
SELECT 
    signup_week, 
    WEEK, 
    returning_users 
FROM weekly_retention
ORDER BY signup_week, WEEK;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
